In [2]:
def obtain_data(file):
    """
    :param file: input data file 
    :return: Dict in the form of {data:[[tweets],[sentiments]], x_set:{tweets}, y_set:{sentiments}}
    """
    data = []
    dictionary = {}
    allKeys = []
    allVals = []
    
    with open(file,'r',encoding="utf8") as f:
        lines = f.readlines()
        #print(len(lines))
        index=0
        for i in range (len(lines)-2000):
            #print(lines[i]) #eg. 'Omg O' is line[0]
            if lines[i] == '\n':
                data.append(lines[index:i]) #append everything until it encounters a \n
                index = i+1
            lines[i] = lines[i].replace('\n','')# replace the \n at the end of each sentiments
            lines[i] = lines[i].split(' ') #split the line into their respective parts  
            #print(lines[i]) ##format = eg. ['Justin', 'B-neutral']
        #convert to keys and values (dict)
        for i in range(len(data)):
            #print(data[i])
            data_values = data[i]
            #print(data_values)
            for j in range(len(data_values)):
                if len(data_values[j]) > 2:
                    #print(data_values[j])
                    for k in range(1, len(data_values[j]) - 1):
                        data_values[j][0] += " "
                        data_values[j][0] += data_values[j][k]
            #key = [data_values[0]]
            #val = [data_values[len(data_values) - 1]]
                #print(data_values[0])
            key = [word[0] for word in data_values] #tweet
            val = [word[-1] for word in data_values] #sentiment
            #print(key)
            #print(val)
            data[i] = [key,val]
            #dictionary = dict(zip(key,val))
            #print(data[i])
        #data[i][0] gives the tweets
        for i in range(len(data)):
            for j in range(len(data[i][0])):
                allVals.append(data[i][1][j]) #appending the sentiments corresponding to the tweets
             
        for i in range(len(data)):
            for j in range(len(data[i][0])):
                #print(data[i][0][j]) #give each word
                allKeys.append(data[i][0][j])
        setKeys = set(allKeys)
        setVals = set(allVals)   

    return dict(data=data,x_set=setKeys,y_set=setVals)


#obtain_data('sg_train')

### Part 2a: (5 pts) Write a function that estimates the emission parameters from the training set using MLE (maximum likelihood estimation):



In [32]:
#e(x|y) = Count(y -> x)/Count(y)
#Count(y->x) means number of times you see x generated from y

import pandas as pd
import numpy as np

def calculate_emission_count(parsed_data):
    """
    :param parsed_data: input preprocessed dataset
    :return: emissions dataframe and count(y)
    ,where emissions dataframe is the count of the sentiments tagged to the tweet whilecount(y) is the number of times the sentiment appears
    """
    data = parsed_data['data']
    x_set = parsed_data['x_set']
    y_set = parsed_data['y_set']
    #create a new datafram of zeros with keys (ie.tweets) as the index and sentiments as the columns
    count_emissions_df = pd.DataFrame(np.zeros((len(x_set),len(y_set))),index=x_set,columns=y_set)
    count_y = pd.Series(np.zeros(len(y_set)),index=y_set) #create a series object of zeros with index as the sentiments => to store the number times the sentiments appear
    #print(count_y)
    #print(count_emissions_df) #datafram structure: where its tweets against columns of sentiments
    
    for word in data:
        #print(word) #format of data => [[keys],[values]]
        #keys are the tweets, values are the sentiments
        tweets_data,sentiments_data = word
        
        for i in range(len(tweets_data)):
            tweet,sentiment = tweets_data[i],sentiments_data[i] #associate the tweet with its sentiment
            #print(tweet,sentiment)
            #print(sentiment)
            #+1 to the row,col, given the tweet, sentiment freq +1
            count_emissions_df.loc[tweet,sentiment] += 1 #.loc[] access a grp of rows and columns by labels
            #count_emissions_df is for Count(y->x) [counting the number of times a sentiment wrt to the tweet]
            count_y[sentiment] += 1 #incrementing the number of time the respective sentiment appear
    return count_emissions_df,count_y
    
def get_emission_params(parsed_data):
    """
    :param parsed_data: input preprocessed dataset
    :return: emissions params 
    """
    count_emissions_df,count_y = calculate_emission_count(parsed_data)
    return count_emissions_df/count_y #e(x|y), where x is the tweet, and y is the sentiment


em_df = get_emission_params(obtain_data('sg_train'))
#get_emission_counts(obtain_data('sg_train'))
em_df.head()


,B-positive,O,B-negative,I-positive,I-neutral,B-neutral,I-negative
https://t.co/OgZ1pIXiSD,0.000000,0.000008,0.0,0.000000,0.00000,0.000000,0.0
ean,0.000000,0.000000,0.0,0.000000,0.00006,0.000000,0.0
Cambridge,0.000154,0.000000,0.0,0.000704,0.00012,0.000168,0.0
Mugen,0.000000,0.000000,0.0,0.000235,0.00000,0.000056,0.0
@eddiesjb_,0.000000,0.000004,0.0,0.000000,0.00000,0.000000,0.0


### Part 2b) (10 pts) One problem with estimating the emission parameters is that some words that appear in the test set do not appear in the training set. One simple idea to handle this issue is as follows. First, replace those words that appear less than k times in the training set with a special token #UNK# before training. This leads to a “modiﬁed training set”. We then use such a modiﬁed training set to train our model. During the testing phase, if the word does not appear in the “modiﬁed training set”, we replace that word with #UNK# as well. Set k to 1, implement this ﬁx into your function for computing the emission parameters. 

In [33]:
def calculate_new_emission_counts(parsed_data,k):
    """
    :param parsed_data: input preprocessed dataset
    :param k:  number of occurrences
    :return: new dataframe with failed tweets replaced with #UNK# and count(y)
    """
    count_emissions_df,count_y = calculate_emission_count(parsed_data)
    #.sum(axis = 1(sum the column), axis = 0 (sum the index))
    count_tweet_appearance = count_emissions_df.sum(axis=1)
    #print(count_tweet_appearance) #counting the number of times each tweet appears by summing everything across the columns
    '''Output of count_tweet appearance eg. 
        seems                      15.0
        https://t.co/h6Ie4IBJ08     1.0
        #AnnaVonHausswolff          2.0
        Bowery                      2.0
        refuge                      2.0
        @chuckielufc                1.0
        https://t.co/7xSNeWemp1     1.0
        @chris_steller              3.0
        unexpected                  3.0
        #usantdp                    1.0
        Ones                        2.0
        1979                        4.0
        @joceltsh                   1.0
        @TomBoxingAsylum            2.0
        @thistletat13               2.0
        @eibeibb                    2.0
        @TalatHussain12             1.0
        Ilkeston                    2.0
        @ricosua                    1.0
        Belarus                     2.0
        charms                      2.0
        @EvermorSolution            2.0
        https://t.co/WrcuWKQ0Xg     2.0
        FIRED                       2.0'''
    
    failed_tweets = count_tweet_appearance[count_tweet_appearance<k]
    #print(failed_tweets)
    '''eg output if k<3 (ie. tweets with occurence less than 3 times) is:
        @Nandos                    1.0
        @rcmpgrcpolice             1.0
        #yas                       1.0
        @just                      2.0
        ford                       1.0
        attracted                  2.0
        @Unitetheunion             1.0
        .....        '''
    
    #replace the tweets that occur less than 1.0 with "#UNK#"
    #print(failed_tweets.index) #gives all the tweets that <1.0
    
    replace_tweets = count_emissions_df.loc[failed_tweets.index].sum(axis=0)
    replace_tweets.name  = '#UNK#'
    
    new_df = count_emissions_df.append(replace_tweets)
    new_df = new_df.drop(failed_tweets.index,axis=0) #drop all failed_tweets words
    headers = new_df.dtypes.index
    #print(headers[3]) #gives sentiment O
    new_df.at['#UNK#',headers[3]] = 1.0
    #print(new_df) #without failed_tweets words inside, has #UNK# row inside at the bottom
    
    return new_df, count_y

def get_new_emission_params(parsed_data,k):
    """
    :param parsed_data: input preprocessed dataset
    :param k: number of occurrences
    :return: new emission params 
    """
    count_emissions_df,count_y = calculate_new_emission_counts(parsed_data,k)
    return count_emissions_df/count_y #e(x|y), where x is the tweet, and y is the sentiment

#calculate_new_emission_counts(obtain_data('sg_train'),3)
new_em_df_parameters = get_new_emission_params(obtain_data('sg_train'),1)
#new_em_df_parameters.sum(axis=1) #gives the sum of each rows (individual respective words)
'''eg.
Throwing          0.000012
headquarters      0.000300
insist            0.000012
except            0.000071
Broken            0.000128
LCD               0.000124
occur             0.000012
sound             0.000071
'''

#new_em_df_parameters.sum(axis=0) #gives the counts of the sentiments
new_em_df_parameters.tail()

,B-positive,O,B-negative,I-positive,I-neutral,B-neutral,I-negative
cow,0.000154,0.000012,0.0,0.000000,0.00000,0.0,0.0
taxonomy,0.000000,0.000008,0.0,0.000000,0.00000,0.0,0.0
Fetus,0.000154,0.000000,0.0,0.000000,0.00000,0.0,0.0
Killeen,0.000000,0.000000,0.0,0.000235,0.00000,0.0,0.0
#UNK#,0.000000,0.000000,0.0,0.000000,0.00006,0.0,0.0


In [95]:
#Note to self: what is axis=1, axis=0 for dataframe pandas
from IPython.display import Image
from IPython.core.display import HTML 
Image(url= "https://i.stack.imgur.com/dcoE3.jpg")

### Part 2c Implement a simple sentiment analysis system that produces the tag => y∗ = argmax e(x|y)



In [ ]:
def training_dataset(file):
    """
    :param file: input preprocessed dataset 
    :return: new emissions params given preprocessed data and k
    """
    dataset = obtain_data(file)
    k = 1
    return get_new_emission_params(dataset,k)

#single sentiment analysis for a word
def sentiment_analysis(emission_param,x):
    """
    :param emission_param: new emissions params dataframe,
    :param x: word that you want to predict the sentiment for 
    :return: predicted sentiment
    """
    #checking if the tweet is an undiscovered/discovered word
    #if the word does not appear in training set, then change it to #UNK#
    #print(emission_param.index) #gives the individual tweets
    if x not in emission_param.index:
        x = '#UNK#'
    probability = emission_param.loc[x,:]
   # print(probability)
    max_probability = None
    for col in probability.index:
        #print(col) #gives the sentiments labels
        '''B-positive
            ..
            ...
            -
            I-negative
            B-neutral
            242
            O
            477
            B-negative
            .
            I-positive
            I-neutral'''
        if max_probability is None : 
            max_probability = probability.loc[col]
            y = col
        elif probability.loc[col]>max_probability: #take the max prob
            max_probability = probability.loc[col]
            y = col #take the sentiment with the highest probability
        
    return y
        

def evaluation(filename,emission_param,outputfile):
    """
    :param filename: input datafile
    :param emission_param: emission param dataframe 
    """
    with open(filename,'r',encoding="utf8") as inputfile:
        lines = inputfile.readlines()
        lines = [line.replace('\n','') for line in lines]
        #print(lines)
        '''['best', 'friends', 'who', 'cry', 'on', 'FaceTime', 'together', ',', 'stay', 'together', '', "I'm", 'at', 'Starbucks',
        'in', 'Johor', 'Bahru', ',', 'Johor', 'w', '/', '@cassiecr17', 'https://t.co/3rzoTtjRag', '', 'Reports', 'of', 'a', 
        'collision', 'on', 'Friary', 'Road', 'in', 'Naas', 'https://t.co/MZgfLNdbyr', '', '♫', 'She', 'Moves', 'In', 'Her', 'Own' ......]
        '''
        
        for i in range(len(lines)):
            line = lines[i] #each individual tweets
            if line != '': #if line is not empty
                line = line + ' ' + sentiment_analysis(emission_param,line)
            line += '\n'
            lines[i] = line
            
        with open(outputfile,"w",encoding="utf8") as outputfile:
            for line in lines:
                outputfile.write(line)
    print("evaluation completed!")
            
        
#testcase below:        
modified_em_params = get_new_emission_params(obtain_data('sg_train'),1)
word = '.'
try:
    print(modified_em_params.loc[word])
except:
    word = '#UNK#'
    print(modified_em_params.loc[word])
sentiment_analysis(modified_em_params,word)
emission_param = training_dataset('SG/train')
evaluation('SG/dev.in',emission_param,'SG/dev.p2.out')

In [36]:
#doing it for all 4 countries
for cty in ["CN","EN","FR","SG"]:
    emission_params = training_dataset(cty+"/train") #new emission params are also obtained inside here
    print("Analysing for " + cty + " dataset:")
    evaluation(cty+"/dev.in",emission_params,cty+"/dev.p2.out")

Analysing for CN dataset:
evaluation completed!
Analysing for EN dataset:
evaluation completed!
Analysing for FR dataset:
evaluation completed!
Analysing for SG dataset:
evaluation completed!


In [41]:
###evalResult.py

import sys
import re
from copy import copy
from collections import defaultdict
from optparse import OptionParser

#Read entities from predcition
def get_predicted(predicted, answers=defaultdict(lambda: defaultdict(defaultdict))):

    example = 0
    word_index = 0
    entity = []
    last_ne = "O"
    last_sent = ""
    last_entity = []

    answers[example] = []
    for line in predicted:
        line = line.strip()
        if line.startswith("##"):
            continue
        elif len(line) == 0:
            if entity:
                answers[example].append(list(entity))
                entity = []

            example += 1
            answers[example] = []
            word_index = 0
            last_ne = "O"
            continue
        else:
            split_line = line.split(separator)
            #word = split_line[0]
            value = split_line[outputColumnIndex]
            ne = value[0]
            sent = value[2:]


            last_entity = []

            #check if it is start of entity
            if ne == 'B' or (ne == 'I' and last_ne == 'O') or (last_ne != 'O' and ne == 'I' and last_sent != sent):
                if entity:
                    last_entity = list(entity)

                entity = [sent]
                    
                entity.append(word_index)

            elif ne == 'I':
                entity.append(word_index)

            elif ne == 'O':
                if last_ne == 'B' or last_ne == 'I':
                    last_entity =list(entity)
                entity = []


            if last_entity:
                answers[example].append(list(last_entity))
                last_entity = []

        last_sent = sent
        last_ne = ne
        word_index += 1

    if entity:
        answers[example].append(list(entity))


    return answers



#Read entities from gold data
def get_observed(observed):


    example = 0
    word_index = 0
    entity = []
    last_ne = "O"
    last_sent = ""
    last_entity = []

    observations=defaultdict(defaultdict)
    observations[example] = []

    for line in observed:
        line = line.strip()
        if line.startswith("##"):
            continue
        elif len(line) == 0:
            if entity:
                observations[example].append(list(entity))
                entity = []

            example += 1
            observations[example] = []
            word_index = 0
            last_ne = "O"
            continue

        else:
            split_line = line.split(separator)
            word = split_line[0]
            value = split_line[outputColumnIndex]
            ne = value[0]
            sent = value[2:]


            last_entity = []

            #check if it is start of entity, suppose there is no weird case in gold data
            if ne == 'B' or (ne == 'I' and last_ne == 'O') or (last_ne != 'O' and ne == 'I' and last_sent != sent):
                if entity:
                    last_entity = entity

                entity = [sent]
                    
                entity.append(word_index)

            elif ne == 'I':
                entity.append(word_index)

            elif ne == 'O':
                if last_ne == 'B' or last_ne == 'I':
                    last_entity = entity
                entity = []


            if last_entity:
                observations[example].append(list(last_entity))
                last_entity = []


        last_ne = ne
        last_sent = sent
        word_index += 1

    if entity:
        observations[example].append(list(entity))

    return observations

#Print Results and deal with division by 0
def printResult(evalTarget, num_correct, prec, rec):
    if abs(prec + rec ) < 1e-6:
        f = 0
    else:
        f = 2 * prec * rec / (prec + rec)
    print('#Correct', evalTarget, ':', num_correct)
    print(evalTarget, ' precision: %.4f' % (prec))
    print(evalTarget, ' recall: %.4f' %   (rec))
    print(evalTarget, ' F: %.4f' % (f))

#Compare results bewteen gold data and prediction data
def compare_observed_to_predicted(observed, predicted):

    correct_sentiment = 0
    correct_entity = 0

    total_observed = 0.0
    total_predicted = 0.0

    #For each Instance Index example (example = 0,1,2,3.....)
    for example in observed:

        if example in discardInstance:
            continue

        observed_instance = observed[example]
        predicted_instance = predicted[example]

        #Count number of entities in gold data
        total_observed += len(observed_instance)
        #Count number of entities in prediction data
        total_predicted += len(predicted_instance)

        #For each entity in prediction
        for span in predicted_instance:
            span_begin = span[1]
            span_length = len(span) - 1
            span_ne = (span_begin, span_length)
            span_sent = span[0]

            #For each entity in gold data
            for observed_span in observed_instance:
                begin = observed_span[1]
                length = len(observed_span) - 1
                ne = (begin, length)
                sent = observed_span[0]

                #Entity matched
                if span_ne == ne:
                    correct_entity += 1


                    #Entity & Sentiment both are matched
                    if span_sent == sent:
                        correct_sentiment += 1

    print()
    print('#Entity in gold data: %d' % (total_observed))
    print('#Entity in prediction: %d' % (total_predicted))
    print()

    prec = correct_entity/total_predicted
    rec = correct_entity/total_observed
    printResult('Entity', correct_entity, prec, rec)
    print()

    prec = correct_sentiment/total_predicted
    rec = correct_sentiment/total_observed
    printResult('Entity Type',correct_sentiment, prec, rec)




##############Main Function##################

if len(sys.argv) < 3:
    print ('Please make sure you have installed Python 3.4 or above!')
    print ("Usage on Windows:  python evalResult.py [gold file] [prediction file]")
    print ("Usage on Linux/Mac:  python3 evalResult.py [gold file] [prediction file]")
    sys.exit()

gold = open('FR\dev.out', "r", encoding='UTF-8')
prediction = open('FR\dev.p2.out', "r", encoding='UTF-8')
discardInstance = []


if len(sys.argv) > 3 and sys.argv[3] == 'filter':
    filterInst_file = open(sys.argv[1] + '.filter', "r", encoding='UTF-8')
    for line in filterInst_file:
        line = line.strip('\n')
        line = line.strip('\r')
        instID = int(line)
        discardInstance.append(instID)


#column separator
separator = ' '

#the column index for tags
outputColumnIndex = 1
#Read Gold data
observed = get_observed(gold)

#Read Predction data
predicted = get_predicted(prediction)

#Compare
compare_observed_to_predicted(observed, predicted)




#Entity in gold data: 238
#Entity in prediction: 1128

#Correct Entity : 186
Entity  precision: 0.1649
Entity  recall: 0.7815
Entity  F: 0.2723

#Correct Entity Type : 78
Entity Type  precision: 0.0691
Entity Type  recall: 0.3277
Entity Type  F: 0.1142


#### >python3 evalResult.py EN/dev.out EN/dev.p2.out
Entity in gold data: 802
Entity in prediction: 926

Correct Entity : 576
Entity  precision: 0.6220
Entity  recall: 0.7182
Entity  F: 0.6667

Correct Entity Type : 507
Entity Type  precision: 0.5475
Entity Type  recall: 0.6322
Entity Type  F: 0.5868

#### >python3 evalResult.py CN/dev.out CN/dev.p2.out
Entity in gold data: 1081
Entity in prediction: 4939

Correct Entity : 613
Entity  precision: 0.1241
Entity  recall: 0.5671
Entity  F: 0.2037

Correct Entity Type : 436
Entity Type  precision: 0.0883
Entity Type  recall: 0.4033
Entity Type  F: 0.1449

#### >python3 evalResult.py FR/dev.out FR/dev.p2.out
Entity in gold data: 238
Entity in prediction: 1128

Correct Entity : 186
Entity  precision: 0.1649
Entity  recall: 0.7815
Entity  F: 0.2723

Correct Entity Type : 78
Entity Type  precision: 0.0691
Entity Type  recall: 0.3277
Entity Type  F: 0.1142

#### >python3 evalResult.py SG/dev.out SG/dev.p2.out
Entity in gold data: 4092
Entity in prediction: 11598

Correct Entity : 2291
Entity  precision: 0.1975
Entity  recall: 0.5599
Entity  F: 0.2920

Correct Entity Type : 1315
Entity Type  precision: 0.1134
Entity Type  recall: 0.3214
Entity Type  F: 0.1676